In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head()

price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                           1.08388                     7.60278   
ethereum                          0.22392                    10.38134   
tether                           -0.21173                     0.04935   
ripple                           -0.37819                    -0.60926   
bitcoin-cash                      2.90585                    17.09717   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                           6.57509                      7.67258   
ethereum                          4.80849                      0.13169   
tether                            0.00640                     -0.04237   
ripple                            2.24984                      0.23455   
bitcoin-cash                     14.75334                     15.74903   

              price_change_percentage_60d  price_change_percentage_200d  \
coin_id                                                                   
bitcoin                          -3.25185                      83.51840   
ethereum                        -12.88890                     186.77418   
tether                            0.28037                      -0.00542   
ripple                          -17.55245                      39.53888   
bitcoin-cash                    -13.71793                      21.66042   

              price_change_percentage_1y  
coin_id                                   
bitcoin                         37.51761  
ethereum                       101.96023  
tether                           0.01954  
ripple                         -16.60193  
bitcoin-cash                    14.49384

In [4]:
# Generate summary statistics
df_market_data.describe()

price_change_percentage_24h  price_change_percentage_7d  \
count                    41.000000                   41.000000   
mean                     -0.269686                    4.497147   
std                       2.694793                    6.375218   
min                     -13.527860                   -6.094560   
25%                      -0.608970                    0.047260   
50%                      -0.063410                    3.296410   
75%                       0.612090                    7.602780   
max                       4.840330                   20.694590   

       price_change_percentage_14d  price_change_percentage_30d  \
count                    41.000000                    41.000000   
mean                      0.185787                     1.545693   
std                       8.376939                    26.344218   
min                     -18.158900                   -34.705480   
25%                      -5.026620                   -10.438470   
50%                       0.109740                    -0.042370   
75%                       5.510740                     4.578130   
max                      24.239190                   140.795700   

       price_change_percentage_60d  price_change_percentage_200d  \
count                    41.000000                     41.000000   
mean                     -0.094119                    236.537432   
std                      47.365803                    435.225304   
min                     -44.822480                     -0.392100   
25%                     -25.907990                     21.660420   
50%                      -7.544550                     83.905200   
75%                       0.657260                    216.177610   
max                     223.064370                   2227.927820   

       price_change_percentage_1y  
count                   41.000000  
mean                   347.667956  
std                   1247.842884  
min                    -17.567530  
25%                      0.406170  
50%                     69.691950  
75%                    168.372510  
max                   7852.089700

In [5]:
df_market_data.shape

(41, 7)

In [6]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [7]:
column_names = df_market_data.columns

# Print the column names
print(column_names)

Index(['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'],
      dtype='object')


In [8]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_scaled = StandardScaler().fit_transform(
    df_market_data[['price_change_percentage_24h',
                    'price_change_percentage_7d', 
                    'price_change_percentage_14d',
                    'price_change_percentage_30d',
                    'price_change_percentage_60d', 
                    'price_change_percentage_200d',
                    'price_change_percentage_1y']]
)
market_scaled[:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [9]:
# Create a DataFrame with the scaled data
df_market_scaled = pd.DataFrame(
    market_scaled,  # Data to be passed to the DataFrame
    columns=['price_change_percentage_24h',
             'price_change_percentage_7d',
             'price_change_percentage_14d',
             'price_change_percentage_30d',
             'price_change_percentage_60d', 
             'price_change_percentage_200d',
             'price_change_percentage_1y'])  # Column names for the DataFrame



# Review the resulting DataFrame
print(df_market_scaled[:3])

# Copy the crypto names from the original data
df_market_scaled["coin_id"] = df_market_data.index
# Set the coinid column as index

df_market_scaled = df_market_scaled.set_index("coin_id")
# Display sample data
df_market_scaled.head()

   price_change_percentage_24h  price_change_percentage_7d  \
0                     0.508529                    0.493193   
1                     0.185446                    0.934445   
2                     0.021774                   -0.706337   

   price_change_percentage_14d  price_change_percentage_30d  \
0                     0.772200                     0.235460   
1                     0.558692                    -0.054341   
2                    -0.021680                    -0.061030   

   price_change_percentage_60d  price_change_percentage_200d  \
0                    -0.067495                     -0.355953   
1                    -0.273483                     -0.115759   
2                     0.008005                     -0.550247   

   price_change_percentage_1y  
0                   -0.251637  
1                   -0.199352  
2                   -0.282061  


price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610                     0.545842   

              price_change_percentage_60d  price_change_percentage_200d  \
coin_id                                                                   
bitcoin                         -0.067495                     -0.355953   
ethereum                        -0.273483                     -0.115759   
tether                           0.008005                     -0.550247   
ripple                          -0.373164                     -0.458259   
bitcoin-cash                    -0.291203                     -0.499848   

              price_change_percentage_1y  
coin_id                                   
bitcoin                        -0.251637  
ethereum                       -0.199352  
tether                         -0.282061  
ripple                         -0.295546  
bitcoin-cash                   -0.270317

---

### Find the Best Value for k Using the Original Data.

In [10]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [11]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(df_market_scaled)
    inertia.append(model.inertia_)    

In [13]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_orin = {
    'k': k,
    'inertia': inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_orin = pd.DataFrame(elbow_data_orin)
df_elbow_orin.head()


k     inertia
0  1  287.000000
1  2  198.571818
2  3  123.190482
3  4   79.022435
4  5   65.302379

In [14]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_orin = df_elbow_orin.hvplot.line(x="k", y="inertia", title="Elbow Curve Using Market Data", xticks=k)
elbow_plot_orin

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Based on the elbow, the best value for k seems to be 2, as it shows a significant reduction in distortion compared to k= 1  and a noticeable but less significant reduction compared to k= 2 onwards.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [15]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=2)

In [16]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data)

KMeans(n_clusters=2)

In [17]:
# Predict the clusters to group the cryptocurrencies using the scaled data
market_clusters = model.predict(df_market_data)

# Print the resulting array of cluster values.
print(market_clusters)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0]


In [18]:
# Create a copy of the DataFrame
df_market_pcp_predict = df_market_data.copy()

In [19]:
# Add a new column to the DataFrame with the predicted clusters
df_market_pcp_predict["Market Clusters"] = market_clusters

# Display sample data
df_market_pcp_predict.head()

price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                           1.08388                     7.60278   
ethereum                          0.22392                    10.38134   
tether                           -0.21173                     0.04935   
ripple                           -0.37819                    -0.60926   
bitcoin-cash                      2.90585                    17.09717   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                           6.57509                      7.67258   
ethereum                          4.80849                      0.13169   
tether                            0.00640                     -0.04237   
ripple                            2.24984                      0.23455   
bitcoin-cash                     14.75334                     15.74903   

              price_change_percentage_60d  price_change_percentage_200d  \
coin_id                                                                   
bitcoin                          -3.25185                      83.51840   
ethereum                        -12.88890                     186.77418   
tether                            0.28037                      -0.00542   
ripple                          -17.55245                      39.53888   
bitcoin-cash                    -13.71793                      21.66042   

              price_change_percentage_1y  Market Clusters  
coin_id                                                    
bitcoin                         37.51761                0  
ethereum                       101.96023                0  
tether                           0.01954                0  
ripple                         -16.60193                0  
bitcoin-cash                    14.49384                0

In [20]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_plot = df_market_pcp_predict.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Market Clusters", # Choose a colormap for coloring the clusters
    hover_cols=["coin_id"] # Add the crypto name in the hover information
)

# Show the scatter plot
scatter_plot

:NdOverlay   [Market Clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [21]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components = 3)

In [22]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_pca_data = pca.fit_transform(df_market_scaled)

# Review the first five rose of the PCA data
# using bracket notation ([0:5])
market_pca_data[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [23]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Total Explained Variance=0.3719856+0.34700813+0.17603793=0.89503166

In [24]:
# Creating a DataFrame with the PCA data
df_market_pca = pd.DataFrame(market_pca_data, columns=["PC1", "PC2",'PC3'])
# Copy the crypto names from the original data

df_market_pca["coin_id"] = df_market_scaled.index
# Set the coinid column as index

df_market_pca = df_market_pca.set_index("coin_id")

# Display sample data
df_market_pca.head()

PC1       PC2       PC3
coin_id                                   
bitcoin      -0.600667  0.842760  0.461595
ethereum     -0.458261  0.458466  0.952877
tether       -0.433070 -0.168126 -0.641752
ripple       -0.471835 -0.222660 -0.479053
bitcoin-cash -1.157800  2.041209  1.859715

---

### Find the Best Value for k Using the PCA Data

In [25]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [26]:
# Create an empty list to store the inertia values
inertia_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model_pca = KMeans(n_clusters = i)
    model_pca.fit(df_market_pca)
    inertia_pca.append(model_pca.inertia_)

inertia_pca

[256.8740855678926,
 165.90199402036018,
 93.77462568057305,
 49.6654966517974,
 38.07279215014322,
 27.720626194899822,
 21.13405603747362,
 17.091636643864756,
 13.668117456693249,
 10.559357562793437]

In [27]:
# Create a dictionary with the data to plot the Elbow curve
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    'k': k,
    'inertia': inertia_pca
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)
df_elbow_pca



k     inertia
0   1  256.874086
1   2  165.901994
2   3   93.774626
3   4   49.665497
4   5   38.072792
5   6   27.720626
6   7   21.134056
7   8   17.091637
8   9   13.668117
9  10   10.559358

In [28]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** From the provided data, the reduction in inertia is significant up to k= 3 and starts to decrease notably after k= 3. Therefore, based on the elbow method analysis, the best value for k when using PCA data appears to be 3.
3



* **Question:** Does it differ from the best k value found using the original data?

  * **Answe For the original data, the best k value differs between the original data and the PCA data. The best k value for the original data is 2, while the best k value for the PCA data is 3. This suggests that the PCA transformation led to a different optimal number of clusters compared to using the original features directly.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [29]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=3)

In [30]:
# Fit the K-Means model using the PCA data
model.fit(df_market_pca)

KMeans(n_clusters=3)

In [31]:
# Predict the clusters to group the cryptocurrencies using the PCA data
market_clusters = model.predict(df_market_pca)
# Print the resulting array of cluster values.
print(market_clusters)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 2
 0 0 0 0]


In [32]:
# Create a copy of the DataFrame with the PCA data
df_market_pca_predictions = df_market_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_market_pca_predictions["MarketCluster"] = market_clusters

# Display sample data
df_market_pca_predictions.head()

PC1       PC2       PC3  MarketCluster
coin_id                                                  
bitcoin      -0.600667  0.842760  0.461595              0
ethereum     -0.458261  0.458466  0.952877              0
tether       -0.433070 -0.168126 -0.641752              0
ripple       -0.471835 -0.222660 -0.479053              0
bitcoin-cash -1.157800  2.041209  1.859715              0

In [33]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="MarketCluster",
    hover_cols=["coin_id"],
    title = "Scatter Plot by Stock Segment - PCA=3"
)

:NdOverlay   [MarketCluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [34]:
# Composite plot to contrast the Elbow curves

In [50]:
import hvplot.pandas as hvplot
import pandas as pd

# Assuming you have computed and stored the elbow data and plots for both original and PCA data

# Composite plot to contrast the Elbow curves
composite_plot = (elbow_plot_orin * elbow_plot_pca).opts(
    title='Elbow Curve Comparison: Original vs PCA Data',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    legend_position='top_right'
)

# Display the composite plot
composite_plot

:Overlay
   .Curve.Original_Data :Curve   [k]   (inertia)
   .Curve.PCA_Data      :Curve   [k]   (inertia)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features through PCA for K-Means clustering had a positive impact. It led to more efficient clustering, lower inertia values, and the ability to identify a clear optimal number of clusters 